This code takes in a claim as a string an first parses it into multiple sentences then for each sentence it asynctronously calls into three datasets to check if the claims are true or false. Then it returns a result

TODO:

Give other ratings besides true or false for a claim(s)

return a link to an article(s) with the true statement(s) // Sally

integrate user feedbck // Sally

Ensure url-safe input. example change wouldn't to would not. change % and $ to words etc.

testing/benchmarks // Daniel

process multiple claims // Sally

https://github.com/ParthaPRay/Ollama_GoogleColab_colabxterm_langchain/blob/main/Ollama_on_Colab_using_xterm_and_langchain.ipynb


https://stackoverflow.com/questions/8551230/how-can-i-get-information-from-an-a-href-tag-within-div-tags-with-beautifuls

https://www.mirascope.com/post/langchain-prompt-template



In [1]:
OLLAMA_MODEL = "llama3.2"
# OLLAMA_MODEL = "llama2"

OLLAMA_HOST = "http://host.docker.internal:11434" # for when running within docker image
# OLLAMA_HOST = "http://localhost:11434"

In [2]:
!pip install -r requirements.txt -q

# !pip install playwright
!apt-get update -y
!apt-get install -y wget curl unzip
!apt-get install -y libx11-dev libxcomposite-dev libxdamage-dev libxrandr-dev libgbm-dev libgtk-3-dev
!playwright install
!playwright install chromium

import requests
import asyncio
import aiohttp
import nest_asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import Ollama
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import nest_asyncio
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
from langchain.prompts import PromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import Ollama

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.18).
unzip is already the newest version (6.0-26ubuntu3.2).
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 74 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libxcomposite-dev is already the newest version (1:0.4.5-1build2).
libxdamage-dev is already the newest version (1:1.1.5-2build2).
libxrandr-dev is already the newest version (2:1.5.2-1build1).
libgbm-dev is already the newest version (23.2.1-1ubuntu3.1~22.04.2).
libgtk-3-dev is already the newe

/opt/conda/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
nest_asyncio.apply()

ollama_model = Ollama(model=OLLAMA_MODEL, base_url=OLLAMA_HOST)

/tmp/ipykernel_4529/3463873494.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  ollama_model = Ollama(model=OLLAMA_MODEL, base_url=OLLAMA_HOST)


In [4]:
ollama_model.invoke("Write a factorial in c code")

'Here\'s an example of how you can implement the factorial function in C:\n\n```c\n#include <stdio.h>\n\n// Function to calculate factorial\nlong long calculateFactorial(int n) {\n    if (n == 0 || n == 1)\n        return 1;\n    \n    else \n        return n * calculateFactorial(n-1);\n}\n\nint main() {\n    int num;\n\n    printf("Enter a number: ");\n    scanf("%d", &num);\n\n    if (num < 0) {\n        printf("Sorry, factorial of negative numbers is not defined.\\n");\n    } else {\n        long long result = calculateFactorial(num);\n        printf("The factorial of %d is %lld\\n", num, result);\n    }\n\n    return 0;\n}\n```\n\nThis code calculates the factorial of a given number. It does this by recursively calling itself until it reaches the base case (i.e., when `n` equals 1), at which point it starts returning values back up the call stack.\n\nPlease note that the size of an integer data type is limited, and can cause issues if you try to calculate large factorials. In such 

In [5]:
async def fetch_article_content(url):
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                response.raise_for_status()
                return await response.text()
    except Exception as e:
        print(f"Error fetching article content from {url}: {e}")
        return ""

async def factcheck_parser(claim):
    claim = claim.replace(" ", "%20")
    url = f"https://www.factcheck.org/search/#gsc.tab=0&gsc.q={claim}&gsc.sort="
    print(url)
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # Launch browser in headless mode
        context = await browser.new_context()
        page = await context.new_page()
        await page.goto(url)
        await page.wait_for_timeout(5000)  # Wait for the page to load

        articles = await page.query_selector_all('div.gs-webResult.gs-result')
        results = []

        for article in articles:
            title_tag = await article.query_selector('a.gs-title')
            if title_tag:
                title = await title_tag.inner_text()
                link = await title_tag.get_attribute('href')
                results.append({'title': title.strip(), 'url': link})

        await browser.close()
        return results

async def retrieve_articles(claim):
    articles = []
    articles.extend(await factcheck_parser(claim))
    return articles

# work in prograss: If no articles were found in any of the three datasets, then use
# newsAPI to try to find articles about the claim
def fetch_news_articles(claim):
    url = (
        'http://newsapi.org/v2/everything?'
        f'q={claim}&'
        'language=en&'
        'sortBy=relevancy&'
        'pageSize=30&'
        'apiKey=4ac92a95346643fdbdb26a7e4d0e98b1'
    )

    try:
        response = requests.get(url)
        response.raise_for_status()  # Throw an error for bad responses
        news_data = response.json()
        return news_data.get('articles', [])
    except Exception as e:
        print(f"Error fetching news articles: {e}")
        return []

from bs4 import BeautifulSoup
import asyncio

async def generate_context_and_assess_claim(claim, context):
    # Fetch news articles related to the claim
    if context == "":
      news_articles = await retrieve_articles(claim)
      print("FactCheck Articles:", news_articles)

      # Combine articles from sources. still to do
      news_articles += fetch_news_articles(claim)
      print("News Articles:", news_articles)

      if news_articles:
          # Limit to top 3 articles
          top_articles = news_articles[:3]

          context += " Here are some recent summaries that provide context:\n"
          for article in top_articles:
              title = article['title']
              url = article['url']

              # Get the article content
              full_content = await fetch_article_content(url)
              description = article.get('description')

              if not description:
                  if full_content:
                      soup = BeautifulSoup(full_content, 'html.parser')
                      first_paragraph = soup.find('p')
                      description = first_paragraph.get_text(strip=True) if first_paragraph else 'No content available.'  

                  else:
                      description = 'No content available.'

              article_context = f"- **{title}**: {description[:200]}... [Read more]({url})\n"  # Truncate to 500 chars
              context += article_context
    
    context_template = f"""
    You are an assistant that provides factual information.
    Analyze the following claim: '{claim}'.
    Context: {context}
    1. State if it is true, false, mostly true, or mostly false.
    2. Provide relevant context or background information.
    3. List key facts and evidence related to this claim.
    4. Mention opposing views or evidence.
    5. If the claim is false, provide the correct information.
    """

    prompt = PromptTemplate(template=context_template, input_variables=["claim"])
    chain = LLMChain(llm=ollama_model, prompt=prompt)

    response = await chain.arun({"claim": claim})

    return response

async def handle_feedback(response, claim, context):
  while True:
      print("\nResponse displayed to the user:")
      print(response)

      rating = input("Please rate the response on a scale of [good, bad, mostly relevant, mostly not relevant]: ").lower()

      feedback = input("Do you have more context or corrections to provide? (y/n): ").lower()
      
      if feedback == 'y':
          additional_context = input("Please provide your additional context or corrections: ")
          context += f"\nAdditional Context: {additional_context}"
      else:
        return

      if rating in ['bad', 'mostly not relevant']:
          print("\nRe-running the LLM chain with updated context...")
          new_response = await generate_context_and_assess_claim(claim, context)
          print("\nUpdated Response based on your feedback:")
          print(new_response)
      else:
          print("Thank you for your feedback! No need for further changes.")

async def main():
    claim = "We would not have left $85 billion worth of brand-new, beautiful military equipment behind"
    claim = input("Please enter a claim to be fact-checked: ")
    context = ""
    result = await generate_context_and_assess_claim(claim, context)
    await handle_feedback(result, claim, context)

if __name__ == "__main__":
    asyncio.run(main())

Please enter a claim to be fact-checked:  We would not have left $85 billion worth of brand-new, beautiful military equipment behind


https://www.factcheck.org/search/#gsc.tab=0&gsc.q=We%20would%20not%20have%20left%20$85%20billion%20worth%20of%20brand-new,%20beautiful%20military%20equipment%20behind&gsc.sort=
FactCheck Articles: [{'title': 'Republicans Inflate Cost of Taliban-Seized U.S. Military Equipment', 'url': 'https://www.factcheck.org/2021/09/republicans-inflate-cost-of-taliban-seized-u-s-military-equipment/'}, {'title': 'FactChecking the Harris-Trump Debate', 'url': 'https://www.factcheck.org/2024/09/factchecking-the-harris-trump-debate/'}, {'title': 'How Many Americans and Allies Are Left in Afghanistan?', 'url': 'https://www.factcheck.org/2021/09/how-many-americans-and-allies-are-left-in-afghanistan/'}, {'title': 'Afghanistan Archives - FactCheck.org', 'url': 'https://www.factcheck.org/location/afghanistan/'}, {'title': "Trump's False Military Equipment Claim - FactCheck.org", 'url': 'https://www.factcheck.org/2020/07/trumps-false-military-equipment-claim/'}, {'title': '', 'url': None}]
News Articles: [{'ti

/tmp/ipykernel_4529/4215792878.py:111: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=ollama_model, prompt=prompt)
/tmp/ipykernel_4529/4215792878.py:113: LangChainDeprecationWarning: The method `Chain.arun` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~ainvoke` instead.
  response = await chain.arun({"claim": claim})



Response displayed to the user:
Based on the provided context, I have analyzed the claim:

**Claim:** "We would not have left $85 billion worth of brand-new, beautiful military equipment behind."

**Classification:** Mostly False

**Context and Background:**
The claim seems to refer to a statement made by President Joe Biden or his administration during their debate with former President Donald Trump. The context suggests that the claim is related to the withdrawal of US forces from Afghanistan in 2021.

**Key Facts and Evidence:**

* According to various sources, including FactCheck.org, the US left behind around $7-8 billion worth of military equipment in Afghanistan, not $85 billion.
* The equipment included vehicles, fuel, ammunition, and other supplies that were not recovered by the Taliban or US forces.
* The equipment was likely left behind due to a combination of factors, including logistical challenges, security concerns, and the rapid pace of withdrawal.

**Opposing Views an

KeyboardInterrupt: Interrupted by user